# Imports

In [1]:
import glob
import os
from itertools import product

import modin.pandas as pd

# import dask.dataframe as pd
import plotly.express as px
import ray
from tqdm.auto import tqdm

ray.shutdown()
ray.init()

2022-07-24 04:39:36,717	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.10.5', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '192.168.31.219', 'raylet_ip_address': '192.168.31.219', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-24_04-39-34_700178_5995/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-24_04-39-34_700178_5995/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-07-24_04-39-34_700178_5995', 'metrics_export_port': 45367, 'gcs_address': '192.168.31.219:52153', 'address': '192.168.31.219:52153', 'node_id': 'a486bdbc9b0f381823f6e901364b19fc2f0b87760fecdf00beeece6d'})

# Determine the correct combination of datasets

Put the data in folder `data`. Extract every file in the folder with the same name. If needed, adapt the path.

In [2]:
tsv_paths = sorted(
    glob.glob("../marijuana19fairman/data/NSDUH_Versions/*/*/*.tsv")
    + glob.glob("../marijuana19fairman/data/NSDUH_Versions/*/*/*/*.tsv")
)
tsv_paths = [
    path
    for path in tsv_paths
    if "Tab" not in path and any(f"{year}.tsv" in path for year in range(2007, 2015))
]

# tsv_paths

In [4]:
work_cols = ["WRKIDST2", "WRKIDSY2"]
filter_cols = ["AGE2", "JBSTATR2"]
survey_cols = ["ANALREC"]
cols = work_cols + survey_cols + filter_cols

In [5]:
version_mapping = {
    "2002-2014": "1",
    "2002-2015": "2",
    "2002-2017": "3",
    "2002-2018": "4",
    "2002-2019": "5",
}

Method: We used yearly cross-sectional data (n = 303,311) from the U.S. National Survey on 
Drug Use and Health (NSDUH) from 2007 through 2014 to compare cigarette smoking trends in 
men and women across rural and urban areas. Current smoking status was modelled using logistic 

Data came from the National Survey on Drug Use and Health (NSDUH)—a U.S. nationally 
representative cross-sectional survey that has been conducted annually since 1990. To ensure 
comparability with Doogan et al. (2017), we restricted our analyses to years 2007 through 
2014, and to adults ( ≥ 18 years). We used participant weights included with the survey data 
to obtain results representative of the US population by correcting for selection probabilities, 

```
AGE2     2   RECODE - FINAL EDITED AGE 
              1 = Respondent is 12 years old........................    2874   5.17 
              2 = Respondent is 13 years old........................    3186   5.73 
              3 = Respondent is 14 years old........................    3139   5.65 
              4 = Respondent is 15 years old........................    3116   5.60 
              5 = Respondent is 16 years old........................    3010   5.41 
              6 = Respondent is 17 years old........................    2969   5.34 
              7 = Respondent is 18 years old........................    2686   4.83 
              8 = Respondent is 19 years old........................    2370   4.26 
              9 = Respondent is 20 years old........................    2255   4.06 
             10 = Respondent is 21 years old........................    2277   4.10 
             11 = Respondent is 22 or 23 years old..................    4545   8.17 
             12 = Respondent is 24 or 25 years old..................    4342   7.81 
             13 = Respondent is between 26 and 29 years old.........    2609   4.69 
             14 = Respondent is between 30 and 34 years old.........    3101   5.58 
             15 = Respondent is between 35 and 49 years old.........    8052  14.48 
             16 = Respondent is between 50 and 64 years old.........    3072   5.52 
             17 = Respondent is 65 years old or older...............    1999   3.60 

```

In [6]:
# numbers = {year: {} for year in range(2007, 2015)}

# for tsv_path in tqdm(tsv_paths):
#     parent_dir = os.path.dirname(tsv_path)
#     with open(os.path.join(parent_dir, "schema.txt"), "r") as f:
#         dtype = eval(f.read())
#         dtype = {k: v if v != int else float for k, v in dtype.items()}
#     df = pd.read_csv(tsv_path, sep="\t", dtype=dtype)

#     year = int(tsv_path.split("/")[-1].removesuffix(".tsv"))
#     version = None
#     for pat, v in version_mapping.items():
#         if pat in tsv_path:
#             version = v
#             break
#     assert version is not None

#     if pd.api.types.is_object_dtype(df["AGE2"].dtype):
#         df["AGE2"] = pd.to_numeric(df["AGE2"].str.split().str[0], errors="coerce")
#         df = df[df["AGE2"].notna()]
#     numbers[year][version] = df[df["AGE2"] <= 7].shape[0]

  0%|          | 0/40 [00:00<?, ?it/s]

ValueError: could not convert string to float: '03/19/2019'

In [ ]:
numbers = [[v for v in vals.values()] for vals in numbers.values()]

from itertools import product

# for combination in product(*numbers):
    if sum(combination) == 303_311:
        print(combination)

In [15]:
# with open(
#     "../marijuana19fairman/data/NSDUH_Versions/NSDUH_Versions/NSDUH-2002-2017-DS0001-bndl-data-tsv/NSDUH_2002_2017_Tab.tsv",
#     "r",
# ) as f:
#     n_of_tabs = [l.count("\t") for l in tqdm(f.readlines())]

# n_of_tabs[:10]

# For now work with the V1 data